In [1]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet, set_random_seed
import joblib

In [2]:
set_random_seed(0)

In [3]:
# import dataser from csv file
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [4]:
# importar ../Data/DatosCicloAprovisionamiento.xlsx
df_ciclo = pd.read_excel('../Data/DatosCicloAprovisionamiento.xlsx')
df_ciclo = df_ciclo.sort_values(by=['sku']).reset_index(drop=True)
df_ciclo = df_ciclo.rename(columns={'sku': 'idSku'})
df_ciclo['ciclo_aprovisionamiento'] = df_ciclo['diasEntrePedidos'] + df_ciclo['diasLeadtime']
df_ciclo.head()

,idSku,diasEntrePedidos,diasLeadtime,ciclo_aprovisionamiento
0,1,18,2,20
1,2,28,4,32
2,3,28,2,30
3,4,14,2,16
4,5,14,4,18


In [5]:
def grid_search(train_1, n_forecasts):
    seasonality_mode = ['multiplicative'] #['multiplicative', 'additive']

    n_changepoints = [10, 100]
    trend_reg = [0] # [0, 0.1]
    n_lags = [1, 7, 14]
    num_hidden_layers = [0, 2]

    # create empty dataframe to store results
    results = pd.DataFrame(columns=['seasonality_mode', 'n_changepoints', 'trend_reg', 'n_lags', 'num_hidden_layers', 'rmse'])
    # grid search
    for mode in seasonality_mode:
        for n_cp in n_changepoints:
            for reg in trend_reg:
                for n_lag in n_lags:
                    for n_layer in num_hidden_layers:
                        m = NeuralProphet(seasonality_mode=mode, n_forecasts=n_forecasts, n_changepoints=n_cp, trend_reg=reg, n_lags=n_lag, num_hidden_layers=n_layer)
                        m.fit(train_1, freq="D")
                        rmse = m.test(train_1)['RMSE'][0]
                        results = results.append({'seasonality_mode': mode, 'n_changepoints': n_cp, 'trend_reg': reg, 'n_lags': n_lag, 'num_hidden_layers': n_layer, 'rmse': rmse}, ignore_index=True)
    return results
    

In [6]:
# create empty dataframe to store best results for each sku
best_results = pd.DataFrame(columns=['idSku', 'seasonality_mode', 'n_changepoints', 'trend_reg', 'n_lags', 'num_hidden_layers', 'rmse'])

In [7]:
# list distinct values of idSku
skus = train['idSku'].unique()
for s in skus:
    train_s = train[train['idSku'] == s]
    train_s = train_s[['idSecuencia', 'udsVenta']]
    train_s = train_s.rename(columns={'idSecuencia': 'ds', 'udsVenta': 'y'})
    n_forecasts = df_ciclo[df_ciclo['idSku'] == s]['ciclo_aprovisionamiento'].values[0]
    results = grid_search(train_s, n_forecasts)
    best_result = results[results['rmse'] == results['rmse'].min()]
    best_result['idSku'] = s
    best_results = best_results.append(best_result, ignore_index=True)
    # fit the model with the parameter values of the best model
    best_m = NeuralProphet(seasonality_mode=best_result['seasonality_mode'].values[0], n_forecasts=n_forecasts, n_changepoints=best_result['n_changepoints'].values[0],
    trend_reg=best_result['trend_reg'].values[0], n_lags=best_result['n_lags'].values[0], num_hidden_layers=best_result['num_hidden_layers'].values[0])
    best_m.fit(train_s, freq="D")
    # save the model
    path = '../Neural_prophet_models/neural_prophet_' + str(s) + '.pkl'
    joblib.dump(best_m, path)


INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.804% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 248
 90%|████████▉ | 107/119 [00:00<00:00, 202.85it/s]
INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.04E-01, min: 1.48E+00
 91%|█████████ | 108/119 [00:00<00:00, 181.43it/s]
INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 3.04E-01, min: 2.10E+00
INFO

In [9]:
best_results

,idSku,seasonality_mode,n_changepoints,trend_reg,n_lags,num_hidden_layers,rmse
0,1,multiplicative,100,0,14,2,7.207204
1,2,multiplicative,100,0,7,2,7.72415
2,3,multiplicative,100,0,14,2,5.886485
3,4,multiplicative,10,0,14,2,4.498294
4,5,multiplicative,100,0,14,2,6.029702
5,6,multiplicative,100,0,1,0,6.660328
6,7,multiplicative,100,0,7,2,4.157397
7,8,multiplicative,100,0,14,2,3.431296
8,9,multiplicative,100,0,7,2,4.517002
9,10,multiplicative,100,0,14,0,4.074048


In [10]:
# save the best results
best_results.to_csv('../Data/NP_best_models.csv', index=False)